In [8]:
import functools
import numpy as np

TIME_HORIZON = 100
EPSILON = 1E-10


class Parameters(object):

    alpha = 0.08
    beta = 0.02
    gamma = 0.7
    sigma = 0.07
    total_cost = 1700.
    partial_payment = 11. * 12
    time_horizon = 20

    names = ['alpha', 'beta', 'gamma', 'sigma', 'total_cost', 'partial_payment', 'time_horizon']

    def __init__(self, **kwargs):
        [setattr(self, n, kwargs[n]) for n in self.names if n in kwargs]
        
    def sigma_correction(self):
        return 1. + 1./self.sigma

    def beta_correction(self):
        return 1. + 1./self.beta

    def accumulated_beta(self):
        return np.power(1. + self.beta, self.time_horizon)

    def accumulated_sigma(self):
        return np.power(1. + self.sigma, self.time_horizon)

    def copy(self, initializers=None):
        p = Parameters()
        for n in self.names:
            new_val = getattr(self, n) if initializers is None or n not in initializers else initializers[n]
            if not np.isscalar(new_val):
                new_val = np.copy(new_val)
            setattr(p, n, new_val)
        return p


def singularitize(func):
    def singularitized_func(params):
        n_scalars = sum(np.isscalar(getattr(params, n)) for n in params.names)
        if n_scalars == len(params.names):
            return func(params)

        if n_scalars < len(params.names) - 1:
            raise ValueError('Weeeeeeeeee')

        iterable_param_name = next(n for n in params.names if not np.isscalar(getattr(params, n)))
        return np.fromiter(map(lambda v: func(params.copy({iterable_param_name: v})),
                               getattr(params, iterable_param_name)), dtype='float')
    return singularitized_func


class Calculator(object):

    @staticmethod
    @singularitize
    def mortgage_cost(params):
        borrowing_money = params.total_cost * params.gamma

        if np.square(params.beta) <= EPSILON:
            return 0.5 * borrowing_money * params.alpha * (params.time_horizon + 1.)

        beta_correction = params.beta_correction()
        alpha_beta_ratio = 1. - params.alpha / params.beta

        periodic_mortgage_stats = (alpha_beta_ratio * beta_correction) / params.time_horizon
        accumulated_beta = params.accumulated_beta()

        vals = (borrowing_money * (-1. + params.alpha * beta_correction)) * accumulated_beta
        vals += borrowing_money * periodic_mortgage_stats * (accumulated_beta - 1.)
        return vals

    @staticmethod
    @singularitize
    def rent_cost(params):
        sigma_correction = params.sigma_correction()
        accumulated_sigma = params.accumulated_sigma()

        if np.square(params.beta) <= EPSILON:
            vals = params.partial_payment * params.time_horizon
            vals -= (params.total_cost * (1. - params.gamma) *
                     np.power(1. + params.sigma, params.time_horizon - 1.))
            return vals

        beta_correction = params.beta_correction()
        accumulated_beta = params.accumulated_beta()
        vals = params.partial_payment * beta_correction * (accumulated_beta - 1)

        if np.square(params.sigma - params.beta) <= EPSILON:
            vals -= (params.total_cost * (1. - params.gamma) *
                     (params.time_horizon * params.beta + 1.) *
                     np.power(1. + params.beta, params.time_horizon - 1.))
        else:
            correlation = (1. + params.beta) / (params.sigma - params.beta)
            vals -= (params.total_cost * (1. - params.gamma) * correlation *
                     (accumulated_sigma / sigma_correction - accumulated_beta / beta_correction))
        return vals

    @staticmethod
    @singularitize
    def buy_cost(params):
        return params.accumulated_beta() - 1

    @staticmethod
    @singularitize
    def flipping_values(params):
        if np.square(params.beta) <= EPSILON:
            vals = 0.5 * params.gamma * params.alpha * (params.time_horizon + 1.)
            vals += (1. - params.gamma) * np.power(1. + params.sigma, params.time_horizon - 1.)
            vals /= params.time_horizon
            return vals

        accumulated_beta = params.accumulated_beta()
        alpha_beta_ratio = 1. - params.alpha / params.beta
        
        if np.square(params.sigma - params.beta) <= EPSILON:
            vals = params.gamma * (params.alpha - 1./params.beta_correction())
            vals += ((1. - params.gamma) * (params.beta * params.time_horizon + 1.) * params.beta /
                     np.square(1 + params.beta))
            vals *= (accumulated_beta / (accumulated_beta - 1))
            vals += params.gamma * alpha_beta_ratio / params.time_horizon
            return vals

        accumulated_sigma = params.accumulated_sigma()
        sigma_beta_ratio = params.beta / (params.sigma - params.beta)
        vals = (params.gamma * ((-1. / params.beta_correction()) + params.alpha +
                                (alpha_beta_ratio / params.time_horizon)) -
                (1. - params.gamma) * sigma_beta_ratio / params.beta_correction()) * accumulated_beta
        vals += accumulated_sigma * (1. - params.gamma) * sigma_beta_ratio / params.sigma_correction()
        vals -= params.gamma * alpha_beta_ratio / params.time_horizon
        vals /= (accumulated_beta - 1.)
        return vals

In [10]:
def point_value():
    params = Parameters(alpha=.08, beta=0.074, gamma=0.65, sigma=0.07, total_cost=1700., 
                        partital_payment=132., time_horizon=np.asarray([10, 20, 30, 40]))
    print('Mortgage: {}'.format(Calculator.mortgage_cost(params)))
    print('Rent: {}'.format(Calculator.rent_cost(params)))
    print('-b/A: {}'.format(Calculator.flipping_values(params)))
    print('Buy cost: {}'.format(Calculator.buy_cost(params)))
    
point_value()

Mortgage: [ 227.96741586  536.08008022 1189.74232733 2561.76734746]
Rent: [   79.01951045   619.87934642   237.18648994 -3469.59154069]
-b/A: [0.08344095 0.07657548 0.08278517 0.09256644]
Buy cost: [ 1.04193923  3.16951583  7.51389796 16.38486228]


In [11]:
from bokeh import plotting
from bokeh import models
plotting.output_notebook()

Loading BokehJS ...

In [12]:
def cost_over_time_horizon():
    params = Parameters(alpha=.08, beta=0.074, gamma=0.65, sigma=0.07, total_cost=1700., 
                        partital_payment=132.,
                        time_horizon= np.arange(1, 100, 1, dtype=float))
    
    p = plotting.figure(title="Cost over T", x_axis_label='T', y_axis_label='Cost (millions)')
    
    p.line(params.time_horizon, Calculator.mortgage_cost(params), legend_label="Mortgage", line_width=2)
    p.line(params.time_horizon, Calculator.rent_cost(params), color='green', legend_label="Rent", line_width=2)
    # p.line(params.time_horizon, Calculator.buy_cost(params), legend_label="Buy", line_width=2)
    plotting.show(p)
    
cost_over_time_horizon()

In [13]:
def flipping_line_over_time_horizon():
    params = Parameters(alpha=.08, beta=0.074, gamma=0.65, sigma=0.07, total_cost=1700., 
                        partital_payment=132.,
                        time_horizon= np.arange(1, 100, 1, dtype=float))
    
    p = plotting.figure(title="-b/A over T", x_axis_label='T', y_axis_label='-b/A')

    p.line(params.time_horizon, Calculator.flipping_values(params), color='green', line_width=2)
    plotting.show(p)
    
flipping_line_over_time_horizon()

In [14]:
def cost_over_beta():
    params = Parameters(alpha=.08, beta=np.arange(-0.99, 2, 0.005, dtype='float'),
                        gamma=0.65, sigma=0.07, total_cost=1700., 
                        partital_payment=132.,
                        time_horizon=20)
    
    p = plotting.figure(title="Cost over beta", x_axis_label='beta', y_axis_label='Cost (millions)')
    
    p.line(params.beta, Calculator.mortgage_cost(params), legend_label='Mortgage', line_width=2)
    p.line(params.beta, Calculator.rent_cost(params), color='green', legend_label="Rent", line_width=2)
    # p.line(params.time_horizon, Calculator.buy_cost(params), legend_label="Buy", line_width=2)
    
    p.y_range = models.Range1d(-2E3, 2E3)
    plotting.show(p)
    
cost_over_beta()